## Dog Breed Classification

In this project we will use traditional CNN, CNN with data augmentation and finally transfer Learning by VGG16 model with weights pre-trained on Imagenet to solve the dog breed classification problem

### Load Dataset Files

In [3]:
# Importing data from Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Importing required packages
import tensorflow as tf
tf.reset_default_graph()
tf.set_random_seed(42)

import numpy as np
from sklearn.model_selection import train_test_split

# Importing keras package
import keras

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Reshape
from keras.layers import Convolution2D, MaxPooling2D, BatchNormalization
from keras.utils import np_utils

from keras import applications
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, EarlyStopping

from keras.optimizers import SGD

Now, upload the given dataset file shared with you in your google drive and give its path for the below given `project_path` variable. For example, a path is given below according to the file path in our google drive. You need to change this to match the path of yours.

In [0]:
# Setting the project path
project_path = '/content/drive/My Drive/AIML_Project/Dog_breed/'

Run the below code to extract all the images in the train.zip files given in the dataset. We are going to use these images as train and validation sets and their labels in further steps.

In [0]:
# Extracting the training data set
from zipfile import ZipFile
with ZipFile(project_path+'train.zip', 'r') as z:
  z.extractall()

Repeat the same step for test.zip

In [0]:
# Extracting the test data set
from zipfile import ZipFile
with ZipFile(project_path+'test.zip', 'r') as z:
  z.extractall()

Repeat the same step for sample_submission.csv.zip

In [0]:
# Extracting the sample submission.csv file
from zipfile import ZipFile
with ZipFile(project_path+'sample_submission.csv.zip', 'r') as z:
  z.extractall()

Repeat the same step for labels.csv.zip

In [0]:
# Extracting the labels.csv file
from zipfile import ZipFile
with ZipFile(project_path+'labels.csv.zip', 'r') as z:
  z.extractall()

After this process, we will have 4 files - Train folder, test folder and labels.csv and sample_submission.csv as part of your google drive

### Read labels.csv file using pandas

In [0]:
import pandas as pd

In [0]:
labels_df = pd.read_csv('labels.csv')

### Print the count of each category of Dogs given in the dataset



In [12]:
labels_df['breed'].value_counts()

scottish_deerhound                126
maltese_dog                       117
afghan_hound                      116
entlebucher                       115
bernese_mountain_dog              114
shih-tzu                          112
pomeranian                        111
great_pyrenees                    111
basenji                           110
samoyed                           109
airedale                          107
tibetan_terrier                   107
leonberg                          106
cairn                             106
japanese_spaniel                  105
beagle                            105
blenheim_spaniel                  102
australian_terrier                102
miniature_pinscher                102
irish_wolfhound                   101
saluki                             99
lakeland_terrier                   99
papillon                           96
whippet                            95
norwegian_elkhound                 95
siberian_husky                     95
pug         

### Get one-hot encodings of labels

In [0]:
#from sklearn.preprocessing import LabelEncoder
#le = LabelEncoder()
#labels_df['breed'] = le.fit_transform(labels_df['breed'])
y = pd.get_dummies(labels_df['breed'])

In [14]:
labels_df.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


## Preparing training dataset
1. Write a code which reads each and every id from labels.csv file and loads the corresponding image (in RGB - 128, 128, 3) from the train folder. <br>
2. Create 2 variables <br> 
     a.  x_train - Should have all the images of the dogs from train folder <br>
     b.  y_train - Corresponding label of the dog <br>
<u>Note:</u> The id of the dog images and its corresponding labels are available in labels.csv file   
<u>Hint:</u> Watch the video shared on "Preparing the training dataset" if you face issue on creating the training dataset

In [15]:
import os
os.listdir()

['.config',
 'sample_submission.csv',
 'labels.csv',
 'test',
 'drive',
 'train',
 'sample_data']

In [0]:
x_train = []
y_train = []
import cv2
for index,row in labels_df.iterrows():
  try:   
      temp = cv2.imread('./train/' + row['id'] + '.jpg')
      temp = cv2.resize(temp,(128,128))
      x_train.append(temp)
      #y_train.append(row['breed'])
  except Exception as e:
      print(e)

In [17]:
x_train[0].shape

(128, 128, 3)

Normalize the training data and convert into 4 dimensions so that it can be used as an input to conv layers in the model

In [0]:
x_train = np.array(x_train)

In [0]:
#Normalize training set
x_train = x_train/255;

In [20]:
x_train.shape

(10222, 128, 128, 3)

### Split the training and validation data from `x_train_data` and `y_train_data` obtained from above step

In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(x_train, y, test_size = 0.2, random_state = 10)

### Loading the test data
Read the id column from the samples_submission.csv and store it in test_img

In [0]:
test_img = pd.read_csv('sample_submission.csv')

In [22]:
test_img.head()

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,000621fb3cbb32d8935728e48679680e,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
1,00102ee9d8eb90812350685311fe5890,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
2,0012a730dfa437f5f3613fb75efcd4ce,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
3,001510bc8570bbeee98c8d80c8a95ec1,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
4,001a5f3114548acdefa3d4da05474c2e,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333


Run the below code to load the test image files in x_test_feature

In [0]:
from tqdm import tqdm
x_test_feature = []
i = 0 # initialisation
for index,row in test_img.iterrows():
    img = cv2.imread('./test/' + row['id'] +'.jpg')
    img_resize = cv2.resize(img, (128, 128)) 
    x_test_feature.append(img_resize)

Normalize the test data and convert it into 4 dimensions

In [0]:
x_test_feature = np.array(x_test_feature)

In [0]:
x_test_feature.shape

(10357, 128, 128, 3)

In [0]:
x_test_feature = x_test_feature/255

### Build a basic conv neural network with 2 conv layers (kernel sizes - 5 and 3) add layers as mentioned below for classification.

1. Add a Dense layer with 256 neurons with `relu` activation

2. Add a Dense layer with 120 neurons as final layer (as there are 120 classes in the given dataset) with `softmax` activation for classifiaction. 

In [22]:
# Define model
model = Sequential()

#Normalizing the data
model.add(BatchNormalization(input_shape=(128, 128, 3)))

# 1st Conv Layer
model.add(Convolution2D(256, (5, 5), input_shape=(128, 128, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

# 2nd Conv Layer
model.add(Convolution2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(128))
model.add(Activation('relu'))

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
# Prediction Layer
model.add(Dense(120))
model.add(Activation('softmax'))

### Use batch_size = 128 and epochs = 10 and execute the model

In [0]:
# Loss and Optimizer
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [26]:
# Train the model
model.fit(X_train, Y_train, batch_size=128, nb_epoch=10, 
        validation_data=(X_test, Y_test))

Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 8177 samples, validate on 2045 samples
Epoch 1/10
8177/8177 [==============================] - 54s 7ms/step - loss: 4.8319 - acc: 0.0086 - val_loss: 4.7864 - val_acc: 0.0098
Epoch 2/10
8177/8177 [==============================] - 42s 5ms/step - loss: 4.7855 - acc: 0.0110 - val_loss: 4.7856 - val_acc: 0.0098
Epoch 3/10
8177/8177 [==============================] - 43s 5ms/step - loss: 4.7841 - acc: 0.0119 - val_loss: 4.7848 - val_acc: 0.0098
Epoch 4/10
8177/8177 [==============================] - 43s 5ms/step - loss: 4.7828 - acc: 0.0119 - val_loss: 4.7842 - val_acc: 0.0098
Epoch 5/10
8177/8177 [==============================] - 42s 5ms/step - loss: 4.7816 - acc: 0.0106 - val_loss: 4.7837 - val_acc: 0.0098
Epoch 6/10
8177/8177 [==============================] - 42s 5ms/step - loss: 4.7806 - acc: 0.0119 - val_loss: 4.7832 - val_acc: 0.0098
Epoch 7/10
8177/8177 [==============================] - 43s 5ms/step - loss: 4.7796 - acc: 0.0119 - val_loss: 4.7829 - val_acc: 0.0098
Epoch 8

In [27]:
# Training accuracy
score = model.evaluate(X_train, Y_train)
print('Training accuracy: ', score[1])

8177/8177 [==============================] - 17s 2ms/step
Training accuracy:  0.01186254127430598


In [28]:
# Test accuracy
score = model.evaluate(X_test, Y_test)
print('Training accuracy: ', score[1])

2045/2045 [==============================] - 4s 2ms/step
Training accuracy:  0.009779951096601184


#The model accuracy is very poor !!!!

### Use Data Augmentation in the above model to see if the accuracy improves


In [0]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    featurewise_center=False,  
    samplewise_center=False, 
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=50,  
    width_shift_range=0.1,  
    height_shift_range=0.1, 
    horizontal_flip=False,  
    vertical_flip=False)


In [0]:
train_datagen.fit(X_train)

In [0]:
val_datagen = ImageDataGenerator(
    featurewise_center=False,  
    samplewise_center=False, 
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=50,  
    width_shift_range=0.1,  
    height_shift_range=0.1, 
    horizontal_flip=False,  
    vertical_flip=False)

In [0]:
val_datagen.fit(X_test)

### Using the above objects, create the image generators with variable names `train_generator` and `val_generator`

You need to use train_datagen.flow() and val_datagen.flow()

In [0]:
train_generator = train_datagen.flow(X_train, Y_train, batch_size=32)

In [0]:
val_generator = val_datagen.flow(X_test, Y_test, batch_size=32)

### Fit the model using fit_generator() using `train_generator` and `val_generator` from the above step with 10 epochs

In [31]:
model.fit_generator(
        train_generator,
        steps_per_epoch=len(X_train) / 32,
        epochs=10,
        validation_data=val_generator,validation_steps = len(X_test) / 32)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
256/255 [==============================] - 61s 240ms/step - loss: 4.7967 - acc: 0.0104 - val_loss: 4.7857 - val_acc: 0.0098
Epoch 2/10
256/255 [==============================] - 55s 214ms/step - loss: 4.7840 - acc: 0.0112 - val_loss: 4.7843 - val_acc: 0.0098
Epoch 3/10
256/255 [==============================] - 55s 213ms/step - loss: 4.7816 - acc: 0.0118 - val_loss: 4.7833 - val_acc: 0.0098
Epoch 4/10
256/255 [==============================] - 55s 214ms/step - loss: 4.7797 - acc: 0.0101 - val_loss: 4.7828 - val_acc: 0.0098
Epoch 5/10
256/255 [==============================] - 54s 213ms/step - loss: 4.7784 - acc: 0.0111 - val_loss: 4.7828 - val_acc: 0.0098
Epoch 6/10
256/255 [==============================] - 55s 213ms/step - loss: 4.7776 - acc: 0.0118 - val_loss: 4.7831 - val_acc: 0.0098
Epoch 7/10
256/255 [==============================] - 55s 213ms/step - loss: 4.7771 - acc: 0.0111 - val_loss: 4.7834 - val_acc: 0.0098
Epoch 8

In [32]:
# Training accuracy
score = model.evaluate(X_train, Y_train)
print('Training accuracy: ', score[1])

8177/8177 [==============================] - 17s 2ms/step
Training accuracy:  0.01186254127430598


In [34]:
# Test accuracy
score = model.evaluate(X_test, Y_test)
print('Test accuracy: ', score[1])

2045/2045 [==============================] - 4s 2ms/step
Test accuracy:  0.009779951096601184


# Model accuracy is still poor!!!

### Lets use Transfer Learning

Download the vgg wieght file from here : https://github.com/MinerKasch/applied_deep_learning/blob/master/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5

Use the below code to load VGG16 weights trained on ImageNet

In [0]:
from keras.applications.vgg16 import VGG16, preprocess_input
# Instantiate the model with the pre-trained weights (no top)
base_model= VGG16(weights=(project_path+'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'),
                 include_top=False, pooling='avg')

Print the summary of the base_model

In [41]:
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

### Add the following classification layers to the imported VGG Model <br>
1. Flatten Layer
2. Dense layer with 1024 neurons with activation as Relu
3. Dense layer with 256 neurons with activation as Relu
4. Dense layer with 120 neurons with activation as Softmax

In [0]:
x = base_model.output
x = Dense(1024, activation="relu")(x)
x = Dense(256, activation="relu")(x)
predictions = Dense(120, activation="softmax")(x)

### Make all the layers in the base_model (VGG16) to be non-trainable

In [0]:
# Freeze the layers which you don't want to train. In this we are freezing the first 5 layers.
for layer in base_model.layers[:5]:
    layer.trainable = False

### Fit and compile the model with batch_size = 128 and epochs = 10 and execute the model

Try to get training and validation accuracy to be more than 90%

In [48]:
# creating the final model 
model_final = Model(input = base_model.input, output = predictions)

sgd = SGD(lr=0.001)

# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = sgd, metrics=["accuracy"])


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  """Entry point for launching an IPython kernel.


ValueError: ignored

In [0]:
# Save the model 
checkpoint = ModelCheckpoint("vgg16_best.h5", monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=1, mode='auto')

epochs=20
# Train the model 
model_final.fit(X_train, Y_train, epochs = epochs, validation_data=(X_test, Y_test), callbacks = [checkpoint, early])